In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from timm import create_model
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 1: Dataset Preparation
data_dir = r"E:\AI\dataset"  # Path to your dataset
assert os.path.exists(data_dir), f"Dataset path does not exist: {data_dir}"

batch_size = 16
input_size = 380  # Input size for EfficientNet-B4

In [ ]:
# Data augmentation and normalization
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize(input_size + 20),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

In [ ]:
# Load dataset using ImageFolder
dataset = datasets.ImageFolder(data_dir)
print(f"Detected class-to-index mapping: {dataset.class_to_idx}")

# Split dataset into training and validation sets
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Apply transforms
train_dataset.dataset.transform = data_transforms["train"]
val_dataset.dataset.transform = data_transforms["val"]

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Step 2: Model Initialization
model = create_model('efficientnet_b4', pretrained=True, num_classes=2)
model = model.to(device)

# Step 3: Define Loss Function, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Step 4: Training Function
def train_model(model, train_loader, val_loader, num_epochs=10, save_dir="model_checkpoints"):
    os.makedirs(save_dir, exist_ok=True)  # Create directory for saving models
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

        scheduler.step()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct.double() / len(train_loader.dataset)

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}")

        # Validate and save the model
        val_acc, val_loss = validate_model(model, val_loader)
        model_save_path = os.path.join(save_dir, f"model_epoch_{epoch + 1}_acc_{val_acc:.4f}.pth")
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved: {model_save_path}")

In [ ]:
# Step 5: Validation Function
def validate_model(model, val_loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = running_loss / len(val_loader.dataset)
    val_acc = correct.double() / len(val_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    cm = confusion_matrix(all_labels, all_preds)
    print(f"Confusion Matrix:\n{cm}")

    return val_acc, val_loss

# Step 6: Train the Model
train_model(model, train_loader, val_loader, num_epochs=10)